In [5]:
import torch
import math
import torch.nn as nn
from torch import Tensor, tensor
from torch.nn.functional import log_softmax
from torch.nn.parallel import DistributedDataParallel as DDP
from beartype import beartype
from jaxtyping import Float, Int, Num, jaxtyped
import copy
from typing import Callable, Tuple
import time

# Transformer

In [6]:
class LayerNorm(nn.Module):
    "Construct a layernorm module"
    
    def __init__(self, d_model: int, eps: float =1e-6):
        # features: number of features == model_dimension
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(d_model))
        self.b_2 = nn.Parameter(torch.zeros(d_model))
        self.eps = eps 

    @jaxtyped(typechecker=beartype)
    def forward(
        self, 
        x: Float[Tensor, "batch_size sequence_length model_dimensions"]
    ) -> Float[Tensor, "batch_size sequence_length model_dimensions"]:
        # subtract and divide by std along feature dimension
        # rescale to learned mean and std
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [7]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model: int, d_ff: int, dropout: float=0.1):
        # d_model: model_dimension
        # d_ff: feed forward hidden layer dimension
        # input d_model, output d_model
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
   
    @jaxtyped(typechecker=beartype)
    def forward(
        self, 
        x: Float[Tensor, "batch_size sequence_length model_dimension"]
    ) -> Float[Tensor, "batch_size sequence_length model_dimension"]:
        return self.w_2(self.dropout(self.w_1(x).relu()))

In [8]:
def clones(module: nn.Module, N: int) -> nn.ModuleList:
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
@jaxtyped(typechecker=beartype)
def attention(
    query: Float[Tensor, "batch_size num_heads subject_sequence_length d_k"], 
    key: Float[Tensor, "batch_size num_heads attend_sequence_length d_k"], 
    value: Float[Tensor, "batch_size num_heads attend_sequence_length d_k"], 
    mask: Float[Tensor, "1 1 subject_sequence_length attend_sequence_length"] = None, 
    dropout = None
) -> Tuple[Float[Tensor, "batch_size num_heads subject_sequence_length d_k"], Float[Tensor, "batch_size num_heads subject_sequence_length attend_sequence_length"]]:
    "Compute `Scaled Dot Product Attention`"
    d_k = query.size(-1)
    scores: Float[Tensor, "batch_size"] = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        # mask along last dimension
         scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [10]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h: int, d_model: int, dropout: float = 0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h 
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None 
        self.dropout = nn.Dropout(p=dropout)
    
    @jaxtyped(typechecker=beartype)
    def forward(
        self, 
        query: Float[Tensor, "batch_size subject_sequence_length model_dimension"], 
        key: Float[Tensor, "batch_size attend_sequence_length model_dimension"], 
        value: Float[Tensor, "batch_size attend_sequence_length model_dimension"], 
        mask: Float[Tensor, "1 subject_sequence_length attend_sequence_length"] = None
    ) -> Tensor:
        if mask is not None:
            # ideally we want mask to be (1, 1, 1, sequence_length)
            # or technically (1, 1, sequence_length, sequence_length)
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
    
        # 1) Do all the linear projections in batch from d_model => h x d_k
        # we have linear layer W and project x to QKV -> still (batch_size sequence_length model_dimension)
        # then we split into different heads -> (batch_size sequence_length num_heads d_k)
        # then we switch num_heads and sequence_length to do multiheadattention -> (batch_size num_heads sequence_length d_k)
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        # 2) Apply attention on all the projected vectors in batch.
        # apply attention on (batch_size num_heads sequence_length d_k)
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )
        
        # 3) "Concat" using a view and apply a final linear.
        # concat heads together after transposing so (batch_size sequence_length num_heads d_k)
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query 
        del key
        del value
        return self.linears[-1](x)

In [11]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    Place these in between attention and feed forward
    """

    def __init__(self, d_model: int, dropout: float):
        # d_model: model_dimension
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(d_model) # LayerNorm with model dimensions
        self.dropout = nn.Dropout(dropout) # Dropout layer 
    
    @jaxtyped(typechecker=beartype)
    def forward(
        self, 
        x: Float[Tensor, "batch_size sequence_length model_dimension"], 
        sublayer
    ) -> Float[Tensor, "batch_size sequence_length model_dimension"]:
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [12]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward"
    
    def __init__(
        self, 
        d_model: int, 
        self_attn: MultiHeadedAttention, 
        feed_forward: PositionwiseFeedForward, 
        dropout: float
    ):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn # attention(query, key, vector, mask)
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(d_model, dropout), 2)
        self.d_model = d_model
    
    @jaxtyped(typechecker=beartype)
    def forward(
        self, 
        x: Float[Tensor, "batch_size src_sequence_length model_dimension"], 
        mask: Float[Tensor, "1 src_sequence_length src_sequence_length"]
    ) -> Float[Tensor, "batch_size src_sequence_length model_dimension"]:
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [13]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    
    def __init__(self, layer: EncoderLayer, N: int):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.d_model)
   
    @jaxtyped(typechecker=beartype) 
    def forward(
        self, 
        x: Float[Tensor, "batch_size src_sequence_length model_dimension"], 
        mask: Float[Tensor, "1 src_sequence_length src_sequence_length"]
    ) -> Float[Tensor, "batch_size src_sequence_length model_dimension"]:
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [14]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src_attn, and feed forward"
    
    def __init__(
        self, 
        d_model: int, 
        self_attn: MultiHeadedAttention, 
        src_attn: MultiHeadedAttention, 
        feed_forward, 
        dropout
    ):
        super(DecoderLayer, self).__init__()
        self.d_model = d_model # model dimension
        self.self_attn = self_attn # attention(query, key, vector, mask)
        self.src_attn = src_attn # attention(query, key, vector, mask)
        self.feed_forward = feed_forward 
        self.sublayer = clones(SublayerConnection(d_model, dropout), 3)
   
    @jaxtyped(typechecker=beartype)
    def forward(
        self,
        x: Float[Tensor, "batch_size tgt_sequence_length model_dimension"], 
        memory: Float[Tensor, "batch_size src_sequence_length model_dimension"], 
        src_mask: Float[Tensor, "1 tgt_sequence_length src_sequence_length"], 
        tgt_mask: Float[Tensor, "1 tgt_sequence_length tgt_sequence_length"]
    ) -> Float[Tensor, "batch_size tgt_sequence_length model_dimension"]:
        m = memory
        # attention(query, key, value, mask)
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [15]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."

    def __init__(self, 
                 layer: DecoderLayer, 
                 N: int):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.d_model)
   
    @jaxtyped(typechecker=beartype)
    def forward(
        self, 
        x: Float[Tensor, "batch_size tgt_sequence_length model_dimension"], 
        memory: Float[Tensor, "batch_size src_sequence_length model_dimension"], 
        src_mask: Float[Tensor, "1 tgt_sequence_length src_sequence_length"], 
        tgt_mask: Float[Tensor, "1 tgt_sequence_length tgt_sequence_length"]
    ) -> Float[Tensor, "batch_size tgt_sequence_length model_dimension"]:
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [16]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step. Used after decoder in transformer"

    def __init__(self, d_model: int, vocab: int):
        super(Generator, self).__init__()
        # (model_dimension, vocab_size)
        self.proj = nn.Linear(d_model, vocab)
   
    @jaxtyped(typechecker=beartype) 
    def forward(
        self, 
        x: Float[Tensor, "batch_size model_dimension"]
    ) -> Float[Tensor, "batch_size vocab_size"]:
        return log_softmax(self.proj(x), dim=-1)

In [17]:
class Embeddings(nn.Module):
    def __init__(self, d_model: int, vocab: int):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model) # vocab_size to model_dimension
        self.d_model = d_model
    
    @jaxtyped(typechecker=beartype)
    def forward(
        self, 
        x: Int[Tensor, "batch_size sequence_length"]
    ) -> Float[Tensor, "batch_size sequence_length model_dimension"]:
        return self.lut(x) * math.sqrt(self.d_model)

In [18]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model: int, dropout: float, max_len: int = 5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)
    
    @jaxtyped(typechecker=beartype)
    def forward(
        self, 
        x: Float[Tensor, "batch_size sequence_length model_dimension"]
    ) -> Float[Tensor, "batch_size sequence_length model_dimension"]:
        x = x + self.pe[:, :x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [19]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """
    
    def __init__(self, 
                 encoder: Encoder, 
                 decoder: Decoder, 
                 src_embed: nn.Sequential, # Embeddings, Positional Encoding
                 tgt_embed: nn.Sequential, # Embeddings, Positional Encoding
                 generator: Generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed 
        self.tgt_embed = tgt_embed
        self.generator = generator
   
    @jaxtyped(typechecker=beartype)
    def forward(
        self,
        src: Int[Tensor, "batch_size src_sequence_length"],
        tgt: Int[Tensor, "batch_size tgt_sequence_length"],
        src_mask: Float[Tensor, "1 src_sequence_length src_sequence_length"],
        tgt_mask: Float[Tensor, "1 tgt_sequence_length tgt_sequence_length"]
    ) -> Float[Tensor, "batch_sequence tgt_sequence_length model_dimension"]:
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
    
    @jaxtyped(typechecker=beartype)
    def encode(
        self, 
        src: Int[Tensor, "batch_size src_sequence_length"], 
        src_mask: Float[Tensor, "1 src_sequence_length src_sequence_length"]
    ) -> Float[Tensor, "batch_size src_sequence_length model_dimension"]:
        return self.encoder(self.src_embed(src), src_mask)
    
    @jaxtyped(typechecker=beartype)
    def decode(
        self, 
        memory: Float[Tensor, "batch_size src_sequence_length model_dimension"], 
        src_mask: Float[Tensor, "1 tgt_sequence_length src_sequence_length"], 
        tgt: Int[Tensor, "batch_size tgt_sequence_length"], 
        tgt_mask: Float[Tensor, "1 tgt_sequence_length tgt_sequence_length"]
    ) -> Float[Tensor, "batch_size tgt_sequence_length model_dimension"]:
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [20]:
def subsequent_mask(size: int) -> Float[Tensor, "1 sequence_length sequence_length"]:
    "Mask out subsequent positions."
    attn_shape = (1, size, size) # (1, sequence_length, sequence_length)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return (subsequent_mask == 0).to(float) # (1, sequence_length, sequence_length)

In [21]:
def make_model(
    src_vocab: int, tgt_vocab: int, N: int = 6, d_model: int = 512, d_ff: int = 2048, h: int = 8, dropout: float = 0.1
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [22]:
def inference_test():
    test_model = make_model(11, 11, 2)
    test_model.eval()
    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    src_mask = torch.ones(1, 10, 10)

    memory = test_model.encode(src, src_mask)
    ys = torch.zeros(1, 1).type_as(src)

    for _ in range(9):
        out = test_model.decode(
            memory, torch.ones(1, ys.size(1), 10), ys, subsequent_mask(ys.size(1))
        )
        prob = test_model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    
    print("Example Untrained Model Prediction:", ys)

def run_tests():
    for _ in range(10):
        inference_test()

run_tests()

torch.Size([1, 1, 10, 10])
torch.Size([1, 1, 10, 10])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1, 10])
torch.Size([1, 1, 1, 1])
torch.Size([1, 1, 1, 10])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 10])
torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 2, 10])
torch.Size([1, 1, 3, 3])
torch.Size([1, 1, 3, 10])
torch.Size([1, 1, 3, 3])
torch.Size([1, 1, 3, 10])
torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 4, 10])
torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 4, 10])
torch.Size([1, 1, 5, 5])
torch.Size([1, 1, 5, 10])
torch.Size([1, 1, 5, 5])
torch.Size([1, 1, 5, 10])
torch.Size([1, 1, 6, 6])
torch.Size([1, 1, 6, 10])
torch.Size([1, 1, 6, 6])
torch.Size([1, 1, 6, 10])
torch.Size([1, 1, 7, 7])
torch.Size([1, 1, 7, 10])
torch.Size([1, 1, 7, 7])
torch.Size([1, 1, 7, 10])
torch.Size([1, 1, 8, 8])
torch.Size([1, 1, 8, 10])
torch.Size([1, 1, 8, 8])
torch.Size([1, 1, 8, 10])
torch.Size([1, 1, 9, 9])
torch.Size([1, 1, 9, 10])
torch.Size([1, 1, 9, 9])
torch.Size([1, 1, 9, 10])
Example Untrained Model Pred

In [23]:
class Batch:
    """Object for holding a batch of data with mask during training."""

    def __init__(self, src, tgt=None, pad=2):
        self.src = src 
        self.src_mask = (src != pad).unsqueeze(-2)
        if tgt is not None:
            self.tgt = tgt[:, :-1]
            self.tgt_y = tgt[:, 1:]
            self.tgt_mask = self.make_std_mask(self.tgt, pad)
            self.ntokens = (self.tgt_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask

In [24]:
class TrainState:
    """Track number of steps, examples, and tokens processed"""

    step: int = 0
    accum_step: int = 0
    samples: int = 0
    tokens: int = 0

In [25]:
def run_epoch(
    data_iter,
    model,
    loss_compute,
    optimizer,
    scheduler,
    mode="train",
    accum_iter=1,
    train_state=TrainState(),
):
    """Train a single epoch"""
    start = time.time()
    total_tokens = 0
    total_loss =0 
    tokens = 0
    n_accum = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(
            batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )
        loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)
        # loss_node = loss_node / accum_iter
        if mode == "train" or mode == "train+log":
            loss_node.backward()
            train_state.step += 1
            train_state.samples += batch.src.shape[0]
            train_state.tokens += batch.ntokens 
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1 
                train_state.accum_step += 1
            scheduler.step()
        
        total_loss += loss 
        total_tokens += batch.ntokens 
        tokens += batch.ntokens 
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]
            elapsed = time.time() - start 
            print(
                (
                    "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
                    + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
                )
                % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
            )
            start = time.time()
            tokens = 0
        del loss 
        del loss_node 
    return total_loss / total_tokens, train_state

In [26]:
def rate(step, model_size, factor, warmup):
    """
    we have to default the step to 1 for LambdaLR function
    to avoid zero raising to negative power.
    """
    if step == 0:
        step = 1 
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )

In [27]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.size = size 
        self.true_dist = None 
    
    def forward(self, x, target):
        assert x.size(1) == self.size 
        true_dist = x.data.clone() 
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist 
        return self.criterion(x, true_dist.clone().detach())

In [28]:
class SimpleLossCompute:
    "A simple loss compute and train function."

    def __init__(self, generator, criterion):
        self.generator = generator 
        self.criterion = criterion 
    
    def __call__(self, x, y, norm):
        x = self.generator(x)
        sloss = (
            self.criterion(
                x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)
            )
            / norm
        )
        return sloss.data * norm, sloss

In [29]:
from datasets import load_dataset

ds = load_dataset("wmt/wmt14", "fr-en")

/opt/miniconda3/envs/lab/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 3003/3003 [00:00<00:00, 440340.33 examples/s]


In [30]:
def data_gen(V, batch_size, nbatches):
    "Generate random data for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.randint(1, V, size=(batch_size, 10))
        data[:, 0] = 1
        src = data.requires_grad_(False).clone().detach()
        tgt = data.requires_grad_(False).clone().detach()
        yield Batch(src, tgt, 0)

In [31]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 2).fill_(start_symbol).type_as(src.data)
    ys[0][1] = 1
    for i in range(max_len - 1):
        out = model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out)
        _, next_word = torch.max(prob, dim=2)
        ys = torch.cat(
            [ys, next_word], dim=1
        )
        return ys

In [32]:
class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

def example_simple_model():
    V = 11
    criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
    model = make_model(V, V, N=2)

    optimizer = torch.optim.Adam(
        model.parameters(), lr=0.5, betas=(0.9, 0.98), eps=1e-9
    )
    from torch.optim.lr_scheduler import LambdaLR
    lr_scheduler = LambdaLR(
        optimizer,
        lr_lambda=lambda step: rate(
            step,
            model_size=model.src_embed[0].d_model,
            factor=1.0,
            warmup=400
        ),
    )

    batch_size = 80
    for epoch in range(20):
        model.train()
        run_epoch(
            data_gen(V, batch_size, 20),
            model,
            SimpleLossCompute(model.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train"
        )
        model.eval()
        run_epoch(
            data_gen(V, batch_size, 5),
            model,
            SimpleLossCompute(model.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval"
        )[0]

        model.eval()
        src = torch.LongTensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
        max_len = src.shape[1]
        src_mask = torch.ones(1, 1, max_len)
        print(greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0))

example_simple_model()

TypeCheckError: Type-check error whilst checking the parameters of __main__.EncoderDecoder.forward.
The problem arose whilst typechecking parameter 'src_mask'.
Actual value: bool[80,1,10](torch)
Expected type: <class 'Float[Tensor, '1 src_sequence_length src_sequence_length']'>.
----------------------
Called with parameters: {
  'self': EncoderDecoder(...),
  'src': i64[80,10](torch),
  'tgt': i64[80,9](torch),
  'src_mask': bool[80,1,10](torch),
  'tgt_mask': bool[80,9,9](torch)
}
Parameter annotations: (self, src: Int[Tensor, 'batch_size src_sequence_length'], tgt: Int[Tensor, 'batch_size tgt_sequence_length'], src_mask: Float[Tensor, '1 src_sequence_length src_sequence_length'], tgt_mask: Float[Tensor, '1 tgt_sequence_length tgt_sequence_length']) -> Any.
The current values for each jaxtyping axis annotation are as follows.
batch_size=80
src_sequence_length=10
tgt_sequence_length=9